In [ ]:
using JuMP
using GLPK
using ECOS
using MAT
using SparseArrays
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using DelimitedFiles
using COSMO

In [ ]:
vars = matread("R3T3.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]
n,d = size(S)
print(size(S))
print(size(L))

n,d = size(S) # n=10 d=100
d,k = size(L) # k = 5

L = vec(L)
U = vec(U)
print("done")

In [ ]:
model = Model(ECOS.Optimizer)

set_optimizer_attribute(model, "max_iter", 500)

@variable(model, L[i] <= x[i = 1:d*k] <= U[i])
@variable(model,  norm_x[1:d])

for i in 1:k
    @constraint(model,S*x[i*d-d+1:i*d] .== 0)
end

for i in 1:d
    @constraint(model,[norm_x[i]; x[[j*d-d+i for j in 1:k]]] in SecondOrderCone())
end
@objective(model,Min,sum(norm_x[i] for i in 1:d))
optimize!(model)

In [ ]:
writedlm( "output.txt", Matrix(reshape(value.(x),(d,k))), ',')